This script calculates the Climate Shelter Index for each public green area.  
A climate shelter should have these characteristics:
- Surface area > 0.5 ha 
- NDVI > 0.4 
- Good accessibility
- Presence of drinking fountains 
- Presence of  benches 
- Presence of picnic tables

### 0. Libraries

In [ ]:
import geopandas as gpd
import pandas as pd

### 1. Climate Shelter Index - CSI

In [9]:
# Load your shapefile
green_area = gpd.read_file(r'D:\Climate_Shelter_Index\isocrona\access_index')

In [10]:
green_area.head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,benches,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry
0,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,9509.007,12461.168,2009-01-29,...,no,N/A,0.542155,0,0,2,3,-1,0.012821,"POLYGON ((11.31509 44.42248, 11.31511 44.42256..."
1,Santo Stefano,GIARDINO MONUMENTO CADUTI DI SABBIUNO,Via di Sabbiuno,PARCO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,5514.205,6730.89,2009-01-29,...,no,N/A,0.459359,0,0,0,0,-1,0.000000,"POLYGON ((11.31288 44.42104, 11.31289 44.42109..."
2,Santo Stefano,PARCO DEI CALANCHI DI SABBIUNO,Via di Sabbiuno,PARCO ESTENSIVO,SOMMITALE,0 -20%,ARGILLOSO COMPATTO,16811.194,24817.745,2007-03-13,...,no,N/A,0.624308,0,0,2,3,-1,0.012821,"POLYGON ((11.31463 44.42146, 11.31496 44.42149..."
3,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,87.272,6250.574,2007-09-20,...,no,N/A,0.877684,0,8,3,8,-1,0.071077,"POLYGON ((11.32362 44.45097, 11.32372 44.45110..."
4,Santo Stefano,PARCO PADERNO,Via Paderno,PARCO ESTENSIVO,PIANO,20 - 40%,MEDIO IMPASTO,-4.817,21360.397,2007-09-20,...,no,N/A,0.879811,0,8,3,8,-1,0.071077,"POLYGON ((11.32160 44.45340, 11.32164 44.45341..."


In [11]:
green_area.columns

Index(['quart', 'nome', 'ubicazione', 'classe_uni', 'classe_gia', 'classe_pen',
       'classe_suo', 'area_prato', 'area_ug', 'data_agg', 'siepi', 'bosco',
       'arboreo', 'arbustivo', 'pavim', 'idro', 'quartiere', 'area_stati',
       'zona_pross', 'area_Ha', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'benches', 'id_bench', 'NDVI', 'n_cycleway', 'n_footway', 'busstop5',
       'busstop10', 'disab_p_5', 'access_ind', 'geometry'],
      dtype='object')

In [12]:
# Define weights for each characteristic (surface area, NDVI, accessibility, drinking fountains, benches, picnic tables)
weights = {
    'surface_area': 0.25,
    'NDVI': 0.25,
    'accessibility': 0.2,
    'drinking_fountains': 0.1,
    'benches': 0.1,
    'picnic_tables': 0.1
}


In [13]:
# Count variables for presence of drinking fountains, benches, and picnic tables
green_area['n_df'] = green_area['id_df'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)
green_area['n_bench'] = green_area['id_bench'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)
green_area['n_pt'] = green_area['id_pt'].apply(lambda x: len(x.split(',')) if x != 'N/A' else 0)


In [14]:
# Define the columns you want to normalize
columns_to_normalize = ['n_df', 'n_bench', 'n_pt']

# Perform min-max normalization for each column
for col_name in columns_to_normalize:
    min_value = green_area[col_name].min()
    max_value = green_area[col_name].max()
    green_area[col_name] = (green_area[col_name] - min_value) / (max_value - min_value)


In [15]:
# Normalize surface area (0.5 ha threshold)
green_area['norm_area'] = green_area['area_Ha'].apply(lambda x: 1 if x >= 0.5 else 0)


In [16]:
# Normalize NDVI (0.4 threshold)
green_area['norm_NDVI'] = green_area['NDVI'].apply(lambda x: x if x >= 0.4 else 0)


In [17]:
# Calculate the CSI based on the weighted and normalized variables
green_area['CSI'] = (
    weights['surface_area'] * green_area['norm_area'] +
    weights['NDVI'] * green_area['norm_NDVI'] +
    weights['accessibility'] * green_area['access_ind'] +
    weights['drinking_fountains'] * green_area['n_df'] +
    weights['benches'] * green_area['n_bench'] +
    weights['picnic_tables'] * green_area['n_pt']
)

In [22]:
green_area = green_area.drop(columns=['n_df', 'n_bench', 'n_pt', 'norm_area', 'norm_NDVI'])

In [23]:
green_area[green_area['NDVI']<0.41].head()

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,geometry,CSI
220,Borgo Panigale - Reno,GIARDINO TERESA NOCE,Via del Beccaccino,GIARDINO,PIANO,NaN,ARGILLOSO COMPATTO,0.114,239.302,2015-06-10,...,N/A,0.398521,0,5,9,34,22,0.125866,"POLYGON ((11.29106 44.49354, 11.29112 44.49352...",0.025173
311,Porto - Saragozza,SCUOLA MEDIA GANDINO,Via Graziano,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,55.935,55.935,2006-06-06,...,N/A,0.388157,0,0,0,0,-1,0.000000,"POLYGON ((11.33103 44.50035, 11.33104 44.50038...",0.000000
317,Porto - Saragozza,SCUOLA ELEM. TERESINA GUIDI,Via Calori,VERDE SCOLASTICO,PIANO,0 -20%,ARGILLOSO COMPATTO,30.760,30.76,2006-06-06,...,N/A,0.378452,0,0,7,38,40,0.102983,"POLYGON ((11.33043 44.50045, 11.33043 44.50046...",0.020597
330,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.783,7.242,2009-10-20,...,N/A,0.351903,0,0,0,0,-1,0.000000,"POLYGON ((11.34252 44.49087, 11.34252 44.49086...",0.000000
332,Santo Stefano,GIARDINO LEGNANI PIZZARDI,Via del Cane 3/b,GIARDINO,PIANO,0 -20%,MEDIO IMPASTO,3.480,6.502,2009-10-20,...,N/A,0.365648,0,0,0,0,-1,0.000000,"POLYGON ((11.34255 44.49084, 11.34256 44.49085...",0.000000


In [ ]:
green_area.to_file(r'D:\Climate_Shelter_Index\isocrona\CSI')

#### 1.2 CSI for selected areas


In [2]:
csi = gpd.read_file(r'D:\Climate_Shelter_Index\isocrona\CSI')
covered_areas = gpd.read_file(r"D:\Meteoblue\covered_areas.shp")

In [3]:
# select only green area within covered areas
covered_areas_csi =csi[csi.geometry.within(covered_areas.unary_union)] 

In [4]:
covered_areas_csi.columns

Index(['quart', 'nome', 'ubicazione', 'classe_uni', 'classe_gia', 'classe_pen',
       'classe_suo', 'area_prato', 'area_ug', 'data_agg', 'siepi', 'bosco',
       'arboreo', 'arbustivo', 'pavim', 'idro', 'quartiere', 'area_stati',
       'zona_pross', 'area_Ha', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'benches', 'id_bench', 'NDVI', 'n_cycleway', 'n_footway', 'busstop5',
       'busstop10', 'disab_p_5', 'access_ind', 'CSI', 'geometry'],
      dtype='object')

In [5]:
covered_areas_csi_sorted = covered_areas_csi.sort_values(by='CSI', ascending=False).head(10)

In [6]:
for index, row in covered_areas_csi_sorted.iterrows():
    print(f"{row['nome']}, with CSI of {round(row['CSI'],3)}")

PARCO DELLA MONTAGNOLA, with CSI of 0.849
GIARDINO CENTRO CIVICO SAN DONATO (EX BENTIVOGLI E MARCINELLE), with CSI of 0.733
PARCO NICHOLAS GREEN ( EX  VILLA CONTRI), with CSI of 0.695
P.CO LUNETTA GAMBERINI, with CSI of 0.694
PARCO DI VILLA ANGELETTI, with CSI of 0.689
GIARDINI MARGHERITA, with CSI of 0.669
PARCO 11 SETTEMBRE 2001 (EX AREA EX MANIFATTURA TABACCHI), with CSI of 0.636
PERCORSO LUNGONAVILE, with CSI of 0.575
CENTRO SPORTIVO CAVINA, with CSI of 0.525
GIARDINO DEL GHISELLO (EX GIARDINO CALDA), with CSI of 0.503


In [7]:
covered_areas_csi_sorted

,quart,nome,ubicazione,classe_uni,classe_gia,classe_pen,classe_suo,area_prato,area_ug,data_agg,...,id_bench,NDVI,n_cycleway,n_footway,busstop5,busstop10,disab_p_5,access_ind,CSI,geometry
488,Santo Stefano,PARCO DELLA MONTAGNOLA,Piazza VIII Agosto,PARCO,PIANO,0 -20%,MEDIO IMPASTO,28445.649,51876.635,2023-02-08,...,"11093356755,11093356866,11093356865,9137608683...",0.723216,0,50,42,69,82,0.632287,0.849095,"MULTIPOLYGON (((11.34654 44.50371, 11.34658 44..."
427,San Donato - San Vitale,GIARDINO CENTRO CIVICO SAN DONATO (EX BENTIVOG...,Piazza Giovanni Spadolini,PARCO,PIANO,0 -20%,MEDIO IMPASTO,9446.168,16567.977,2010-07-23,...,"11093642583,11093642585,11093642584,1109364254...",0.651157,1,12,12,45,22,0.201932,0.732509,"POLYGON ((11.36568 44.50428, 11.36583 44.50436..."
185,Borgo Panigale - Reno,PARCO NICHOLAS GREEN ( EX VILLA CONTRI),Via della Certosa,PARCO,PIANO,0 -20%,MEDIO IMPASTO,50456.763,58927.621,2020-09-29,...,"10896934099,10896934098,3180723125,3180723124,...",0.684428,10,14,10,41,10,0.261080,0.695323,"POLYGON ((11.30137 44.49696, 11.30138 44.49699..."
835,Santo Stefano,P.CO LUNETTA GAMBERINI,"Via degli Orti 60, Largo Lercaro, Via Nadi",PARCO,PIANO,0 -20%,ARGILLOSO COMPATTO,64346.153,81519.19,2022-03-23,...,"10676938409,10676938435,10676938434,1067693843...",0.669272,18,10,19,51,146,0.444982,0.693981,"POLYGON ((11.37171 44.48198, 11.37261 44.48164..."
502,Navile,PARCO DI VILLA ANGELETTI,Via Carracci,PARCO,PIANO,0 -20%,ARGILLOSO COMPATTO,59536.660,89013.989,2007-05-08,...,"1653184518,1653184559,1653184550,1653184647,16...",0.770084,11,4,13,45,19,0.233576,0.689236,"POLYGON ((11.33405 44.51449, 11.33406 44.51430..."
874,Santo Stefano,GIARDINI MARGHERITA,Viale Gozzardini,PARCO,PIANO,0 -20%,MEDIO IMPASTO,149344.335,203169.402,2007-03-14,...,"6328825142,8990334463,8990334458,8990334459,89...",0.690379,0,10,25,63,32,0.270727,0.668574,"MULTIPOLYGON (((11.35401 44.48366, 11.35402 44..."
322,Porto - Saragozza,PARCO 11 SETTEMBRE 2001 (EX AREA EX MANIFATTUR...,Via Rivareno,PARCO,PIANO,0 -20%,MEDIO IMPASTO,10772.264,14073.304,2006-06-11,...,"11107786706,11107772107,11107772108,1110778860...",0.682335,0,0,14,61,68,0.180638,0.636378,"POLYGON ((11.33618 44.50023, 11.33630 44.50023..."
527,Navile,PERCORSO LUNGONAVILE,Via Yuri Gagarin - Via dei Terraioli,PARCO,PIANO,0 -20%,MEDIO IMPASTO,8205.678,10826.838,2007-09-13,...,469864580,0.756890,39,19,26,91,29,0.658218,0.575033,"POLYGON ((11.33525 44.52143, 11.33525 44.52143..."
62,Borgo Panigale - Reno,CENTRO SPORTIVO CAVINA,via Biancolelli,VERDE SPORTIVO,PIANO,0 -20%,MEDIO IMPASTO,12697.181,17690.31,2005-08-22,...,"6316104034,6316104033,6316108901,6316105837,63...",0.705562,0,8,2,12,23,0.087037,0.525465,"POLYGON ((11.26946 44.52346, 11.26947 44.52346..."
242,Borgo Panigale - Reno,GIARDINO DEL GHISELLO (EX GIARDINO CALDA),via Treves,GIARDINO,PIANO,NaN,ARGILLOSO COMPATTO,5036.162,5797.385,2015-04-15,...,"11032970081,11032970075,11032970072,1103297007...",0.758808,4,8,16,32,32,0.212574,0.503050,"POLYGON ((11.29746 44.49094, 11.29746 44.49094..."


In [8]:
covered_areas_csi.to_file(r'D:\Climate_Shelter_Index\CSI_covered_areas')

##### 1.2.1 Green Surface > 0.5

In [9]:
covered_areas_csi[covered_areas_csi['area_Ha']>0.5].shape

(181, 35)

In [10]:
# find the top 10 parks with the largest surface area
covered_areas_csi_sorted_2 = covered_areas_csi.sort_values(by='area_Ha', ascending=False).head(10)

In [11]:
for index, row in covered_areas_csi_sorted_2.iterrows():
    print(f"{row['nome']}, with a green area of {round(row['area_Ha'], 3)} hectares")

GIARDINI MARGHERITA, with a green area of 14.934 hectares
LUNGO RENO CHIARINI BERTOCCHI, with a green area of 8.194 hectares
PARCO VINCENZO TANARA, with a green area of 6.644 hectares
P.CO LUNETTA GAMBERINI, with a green area of 6.435 hectares
PARCO BADEN POWELL, with a green area of 6.123 hectares
PARCO DI VILLA ANGELETTI, with a green area of 5.954 hectares
PARCO DELLE QUERCE, with a green area of 5.446 hectares
PARCO NICHOLAS GREEN ( EX  VILLA CONTRI), with a green area of 5.046 hectares
LUNGORENO PONTE BACCHELLI - PONENTE, with a green area of 4.755 hectares
LUNGORENO TRATTO TRIUMVIRATO, with a green area of 4.412 hectares


##### 1.2.2 NDVI > 0.4

In [12]:
covered_areas_csi[covered_areas_csi['NDVI']>0.4].shape

(702, 35)

##### 1.2.3 NDVI + Green Surface

In [13]:
covered_areas_csi[(covered_areas_csi['NDVI']>0.4) & (covered_areas_csi['area_Ha']>0.5)].shape

(181, 35)